### <center> Vertical Correction for Oblique UAV Image using OpenCV and NumPy </center>

##### Reference Paper: https://www.researchgate.net/publication/349184942_Video_Stitching_method_for_a_Surveillance_System_Deployed_at_a_Fully_Mechanized_Mining_Face


In [1]:
import cv2
import numpy as np
import os
import pandas as pd

In [2]:
folders = ["S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230823_Route9-Orchard1perp_0.39GSD/"]
        #    "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230824_Route9-Orchard1perp_0.84GSD/",
        #    "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230824_Route9-Orchard2_0.86GSD/",
        #    "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/20230824_Route9-Orchard3-closePar_RGB_0.67GSD/",
        #    "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/DJI_202308231046_004_Route9Orchard1South_0.75GSD/",
        #    "S:/Zack/Imagery/Chestnut/Annotation/Ohio/Route9/DJI_202308231102_005_Route9Orchard1North_1.60GSD/"]

for folder in folders:

    # Get list of .csv files in folder
    files = [file for file in os.listdir(folder) if file.endswith((".csv"))]

    # loop through files to load into dataframe
    for file in files:
        # load csv file into dataframe
        df = pd.read_csv(folder + file)
    
    # loop through images and load any oblique images using df['SourceFile'] as path for loading image
    for image in df.index:
        
        # perform vertical correction if image is oblique
        if abs(df['GimbalPitchDegree'][image].astype(float)) != 90:
            
            # get path to image
            path = df['SourceFile'][image]

            # load image
            img = cv2.imread(path)

            # get image dimensions
            img_H = img.shape[0]
            img_W = img.shape[1]

            # alpha is angle of rotation from nadir (vertical; 90 degrees) to oblique
            alpha = 90 - abs(df['GimbalPitchDegree'][image].astype(float))
            # print('alpha: ', alpha)
            # print()

            # f is focal length of camera
            f = df['FocalLength'][image].astype(float)

            # create the two mapping matrices for x and y
            map_x = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)
            map_y = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)
            
            # for every pixel in img, correct for obliqueness
            for x in range(map_x.shape[0]):
                
                for y in range(map_x.shape[1]):
                    
                    # Store pixel coordinates in matrix with 1 column 3 rows
                    pixel = np.array([[x],[y],[f]], dtype = float)
                    # print('pixel: ', pixel)
                    # print()

                    # lambda_pixel equals f divided by y*sin(alpha)+x*cos(alpha)
                    lambda_pixel = f / (pixel[1]*np.sin(alpha) + pixel[0]*np.cos(alpha))
                    # print('lambda: ', lambda_pixel)
                    # print()

                    # define perspective correction matrix following paper linked above
                    transformation_matrix = np.array([[1, 0, 0], 
                                                    [0, np.cos(alpha), -np.sin(alpha)], 
                                                    [0, np.sin(alpha), np.cos(alpha)]])
                    # print('transformation matrix: ', transformation_matrix)
                    # print()

                    # pixel' equals lambda_pixel * [transformation_matrix] * [pixel]
                    pixel_prime = np.matmul(transformation_matrix, pixel) * lambda_pixel 
                    # print('corrected pixel: ', pixel_prime)
                    # print()

                    # create a map that correlates pixel' to pixel
                    map_x[x,y] = pixel_prime[0]
                    map_y[x,y] = pixel_prime[1]
                    # print('map_x: ', map_x[x,y])
                    # print()

            # remap image using map_x and map_y
            dst = cv2.remap(img, map_x, map_y, cv2.INTER_LINEAR)

            # save image in same folder as original image. append 'corrected' to filename
            cv2.imwrite(path[:-4] + '_corrected.jpg', dst)

        # break after first image
        break
    break

C:\Users\zack\AppData\Local\Temp\ipykernel_29868\683894077.py:57: RuntimeWarning: divide by zero encountered in divide
  lambda_pixel = f / (pixel[1]*np.sin(alpha) + pixel[0]*np.cos(alpha))
C:\Users\zack\AppData\Local\Temp\ipykernel_29868\683894077.py:69: RuntimeWarning: invalid value encountered in multiply
  pixel_prime = np.matmul(transformation_matrix, pixel) * lambda_pixel


In [3]:
# display corrected image
cv2.imshow('corrected', dst)